# Coding Homework W10/11 Submission by Ng Jia Sheng Jason 1002749

## Question 1

In [30]:
# load necessary packages
import os
import torch
import matplotlib.pyplot as plt
import numpy as np
from torchvision import datasets, transforms
import guidedbpcodehelpers
from imgnetdatastuff import dataset_imagenetvalpart
from PIL import Image
import glob
import pickle

In [2]:
# load and transform data
data_dir = r"C:\Users\Jason\Desktop\SUTD\Term 8 Last Term\AI\Homework\Week 10\mnt\scratch1\data\imagespart"

data_transform = transforms.Compose([transforms.RandomResizedCrop(224),
                                     transforms.ToTensor(),
                                     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

trainloader = dataset_imagenetvalpart(root_dir = data_dir, maxnum = 0, transform = data_transform)

In [3]:
#device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [25]:
# load pre-trained model
model1 = torch.hub.load('pytorch/vision:v0.6.0', 'vgg16', pretrained=True)
model2 = torch.hub.load('pytorch/vision:v0.6.0', 'vgg16_bn', pretrained=True)

Using cache found in C:\Users\Jason/.cache\torch\hub\pytorch_vision_v0.6.0
Using cache found in C:\Users\Jason/.cache\torch\hub\pytorch_vision_v0.6.0


In [26]:
# implement parameterized backward hook
class backward_hook():
    def __init__(self, module):
        self.hook = module.register_backward_hook(self.hook_fn)
    
    def hook_fn(self, module, grad_input, grad_output):
        self.norm = grad_output[0].norm()
    
    def close(self):
        self.hook.remove()

# register hooks in vgg16
vgg16_hooks = [backward_hook(model1.features[0]), backward_hook(model1.features[2])]

# register hooks in vgg16_bn
vgg16_hooks_bn = [backward_hook(model2.features[0]), backward_hook(model2.features[2])]

In [24]:
print(os.path.basename(trainloader[0]['filename']).replace(".JPEG",""))

ILSVRC2012_val_00000001


In [ ]:
# run backprog and get norms, save them (for each model)
for i in range(0, 251):
    image = trainloader[i]["image"].unsqueeze(0)
    name = os.path.basename(trainloader[i]['filename']).replace(".JPEG","")
    
    # for vgg16
    out1 = model1(image)
    gradients = torch.ones_like(out1)
    out1.backward(gradients)
    
    with open(r'C:\Users\Jason\Desktop\SUTD\Term 8 Last Term\AI\Homework\Week 10\mnt\scratch1\data\vgg16_grad/' + name + '.pkl' , 'wb') as f:
        pickle.dump(vgg16_hooks, f)
    
    #for vgg16_bn
    out2 = model2(image)
    gradients = torch.ones_like(out2)
    out2.backward(gradients)
    
    with open(r'C:\Users\Jason\Desktop\SUTD\Term 8 Last Term\AI\Homework\Week 10\mnt\scratch1\data\vgg16_bn_grad/' + name + '.pkl' , 'wb') as g:
        pickle.dump(vgg16_hooks_bn, g)
    

In [27]:
print(vgg16_hooks[0].norm)

AttributeError: 'backward_hook' object has no attribute 'norm'

## Question 2